# pyphot - Interface with SVO filter profile service

[http://svo2.cab.inta-csic.es/theory/fps/](http://svo2.cab.inta-csic.es/theory/fps/)

If your research benefits from the use of the SVO Filter Profile Service, include the following acknowledgement in your publication:

> This research has made use of the SVO Filter Profile Service
> (http://svo2.cab.inta-csic.es/theory/fps/) supported from the Spanish MINECO
> through grant AYA2017-84089.

and please include the following references in your publication:

* The SVO Filter Profile Service. Rodrigo, C., Solano, E., Bayo, A., 2012; https://ui.adsabs.harvard.edu/abs/2012ivoa.rept.1015R/abstract
* The SVO Filter Profile Service. Rodrigo, C., Solano, E., 2020; https://ui.adsabs.harvard.edu/abs/2020sea..confE.182R/abstract


Some examples are provided in this notebook

Full documentation available at https://mfouesneau.github.io/pyphot/

In [1]:
%matplotlib inline

import pylab as plt
import numpy as np

import sys
sys.path.append('../')
from pyphot import astropy as pyphot
from pyphot.svo import get_pyphot_astropy_filter as get_filter_from_svo

## Quick Start

Quick start example to access the library and it's content

In [2]:
lst = ["2MASS/2MASS.J", "2MASS/2MASS.H", "2MASS/2MASS.Ks",
       "HST/ACS_WFC.F475W", "HST/ACS_WFC.F814W"]
filters = [get_filter_from_svo(k) for k in lst]
filters

[Filter: 2MASS_2MASS.J, <pyphot.astropy.sandbox.UnitFilter object at 0x12737c790>,
 Filter: 2MASS_2MASS.H, <pyphot.astropy.sandbox.UnitFilter object at 0x1053d8730>,
 Filter: 2MASS_2MASS.Ks, <pyphot.astropy.sandbox.UnitFilter object at 0x12737cf10>,
 Filter: HST_ACS_WFC.F475W, <pyphot.astropy.sandbox.UnitFilter object at 0x1281a0850>,
 Filter: HST_ACS_WFC.F814W, <pyphot.astropy.sandbox.UnitFilter object at 0x1281c46d0>]

In [3]:
# get the internal default library of passbands filters
filters[0].info(show_zeropoints=True)

Filter object information:
    name:                 2MASS_2MASS.J
    detector type:        energy
    wavelength units:     nm
    central wavelength:   1241.051880 nm
    pivot wavelength:     1235.808838 nm
    effective wavelength: 1228.565473 nm
    photon wavelength:    1232.122857 nm
    minimum wavelength:   1082.000000 nm
    maximum wavelength:   1406.000000 nm
    norm:                 162.432434
    effective width:      162.432434 nm
    fullwidth half-max:   3.000000 nm
    definition contains 107 points

    Zeropoints
        Vega: 23.756942 mag,
              3.1421235331555586e-10 erg / (Angstrom cm2 s),
              1600.6823123157903 Jy
              189.9448327751629 ph / (Angstrom cm2 s)
          AB: 22.867705 mag,
              7.127186272302632e-10 erg / (Angstrom cm2 s),
              3630.7805477009983 Jy
          ST: 21.100000 mag,
              3.6307805477010028e-09 erg / (Angstrom cm2 s),
              18496.173499482553 Jy
        


Suppose one has a calibrated spectrum and wants to compute the vega magnitude throug the HST WFC3 F110W passband,

In [5]:
# convert to magnitudes
import numpy as np

# We'll use Vega spectrum as example
from pyphot.astropy import Vega
vega = Vega()
f = filters[-1]
# compute the integrated flux through the filter f
# note that it work on many spectra at once
fluxes = f.get_flux(vega.wavelength, vega.flux, axis=-1)
# Note that fluxes is now with units of erg/s/cm2/AA
# pyphot gives Vega in flam and can convert between flux density units. 
fluxes, vega.wavelength, vega.flux

(<Quantity 1.14049113e-09 erg / (Angstrom cm2 s)>,
 <Quantity [9.00452026e+02, 9.01354004e+02, 9.02257996e+02, ...,
            2.99353200e+06, 2.99653275e+06, 2.99953700e+06] Angstrom>,
 <Quantity [1.23800003e-17, 1.67599994e-17, 1.78000003e-17, ...,
            1.40099994e-19, 1.38700004e-19, 1.26499994e-19] flam>)

In [6]:
# convert to vega magnitudes
mags = -2.5 * np.log10(fluxes.value) - f.Vega_zero_mag
print("Vega magnitude of Vega in {0:s} is : {1:f} mag".format(f.name, mags))
mags = -2.5 * np.log10(fluxes.value) - f.AB_zero_mag
print("AB magnitude of Vega in {0:s} is : {1:f} mag".format(f.name, mags))
mags = -2.5 * np.log10(fluxes.value) - f.ST_zero_mag
print("ST magnitude of Vega in {0:s} is : {1:f} mag".format(f.name, mags))

Vega magnitude of Vega in HST_ACS_WFC.F814W is : 0.000000 mag
AB magnitude of Vega in HST_ACS_WFC.F814W is : 0.421694 mag
ST magnitude of Vega in HST_ACS_WFC.F814W is : 1.257270 mag
